In [16]:
import pandas as pd
from nba_api.stats.endpoints import commonplayerinfo
from time import sleep

In [17]:
TEAMS_CSV_DIR = "./data/csv/teams"

heights_not_in_api = {
    # Tyler Hansbrough
    201946: "6-9"
}

def get_height_for_player(player_row):
    id = player_row["playerid"]
    # NBA API will block the IP if we request too quickly
    # We need to wait 600ms between requests
    # https://github.com/swar/nba_api/issues/176#issuecomment-771991604
    sleep(0.6)
    player_info = commonplayerinfo.CommonPlayerInfo(player_id=id)
    height_str = player_info.get_normalized_dict()["CommonPlayerInfo"][0]["HEIGHT"]
    if height_str.strip() == "":
        if id in heights_not_in_api:
            height_str = heights_not_in_api[id]
        else:
            raise ValueError(f"No height info for player {id}")
    height_ft, height_in = height_str.split("-")
    height = int(height_ft) * 12 + int(height_in)
    return height

def get_stats_for_team(team_df: pd.DataFrame):
    if "height" not in team_df.columns:
        team_df["height"] = team_df.apply(get_height_for_player, axis="columns")
    return team_df

In [18]:
path = f"{TEAMS_CSV_DIR}/PHI.csv"
print(path)
team_df = pd.read_csv(path)
team_df = get_stats_for_team(team_df)
team_df

./data/csv/teams/PHI.csv


,name,teamid,abbreviation,lastname,firstname,playerid,jersey,position,height
0,Philadelphia 76ers,1610612755,PHI,Canaan,Isaiah,203477,0.0,G,72
1,Philadelphia 76ers,1610612755,PHI,Smith,Ish,202397,NaN,G,72
2,Philadelphia 76ers,1610612755,PHI,Noel,Nerlens,203457,4.0,F-C,82
3,Philadelphia 76ers,1610612755,PHI,Marshall,Kendall,203088,5.0,G,76
4,Philadelphia 76ers,1610612755,PHI,Landry,Carl,201171,7.0,F,81
5,Philadelphia 76ers,1610612755,PHI,Sampson,JaKarr,203960,9.0,G-F,79
6,Philadelphia 76ers,1610612755,PHI,Stauskas,Nik,203917,11.0,G,78
7,Philadelphia 76ers,1610612755,PHI,McConnell,TJ,204456,12.0,G,73
8,Philadelphia 76ers,1610612755,PHI,Holmes,Richaun,1626158,22.0,F,80
9,Philadelphia 76ers,1610612755,PHI,Thompson,Hollis,203138,31.0,G-F,80
